GENERAL
============================= 

Desarrollar las siguientes consultas SQL e incluir en un documento una captura de pantalla con la salida de cada una, junto con una breve interpretación. Además, entregar el script SQL completo utilizado.

- Bajar datos CSV.
- Habilitar en edit_conections (advanced) la lectura de rutas absolutas en maq. local.
- Cambiar Path de ruta de archivos segun sea el caso (EJ: LOAD DATA LOCAL INFILE = 'C:\\Users\\nicol\\OneDrive\\Desktop\\').


___

TASK 1 (Load Data)
============================= 

``` 
USE mi_base_de_datos;
SET SQL_SAFE_UPDATES = 0;

# ----------------> Countries Table
DROP TABLE IF EXISTS countries;
CREATE TABLE countries (
    CountryID   INT,
    CountryName VARCHAR(45),
    CountryCode VARCHAR(2)
    );

LOAD DATA LOCAL INFILE 'C:\\Users\\nicol\\OneDrive\\Desktop\\countries.csv'
INTO TABLE countries
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


# ----------------> Cities Table
DROP TABLE IF EXISTS cities;
CREATE TABLE cities (
    CityID   INT,
    CityName VARCHAR(45),
    Zipcode  VARCHAR(10),
    CountryID INT
);
LOAD DATA LOCAL INFILE 'C:\\Users\\nicol\\OneDrive\\Desktop\\cities.csv'
INTO TABLE cities
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


# ----------------> Customers Table
DROP TABLE IF EXISTS customers;
CREATE TABLE customers (
    CustomerID    INT,
    FirstName     VARCHAR(45),
    MiddleInitial VARCHAR(1),
    LastName      VARCHAR(45),
    CityID        INT,
    Address       VARCHAR(90)
);
LOAD DATA LOCAL INFILE 'C:\\Users\\nicol\\OneDrive\\Desktop\\customers.csv'
INTO TABLE customers
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


# ----------------> Employees Table
DROP TABLE IF EXISTS employees;
CREATE TABLE employees (
    EmployeeID    INT,
    FirstName     VARCHAR(45),
    MiddleInitial VARCHAR(1),
    LastName      VARCHAR(45),
    BirthDate     VARCHAR(20),
    Gender        VARCHAR(1),
    CityID        INT,
    HireDate      VARCHAR(100)
);
LOAD DATA LOCAL INFILE 'C:\\Users\\nicol\\OneDrive\\Desktop\\employees.csv'
INTO TABLE employees
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


# ----------------> Categories Table
DROP TABLE IF EXISTS categories;
CREATE TABLE categories (
    CategoryID   INT,
    CategoryName VARCHAR(45)
);
LOAD DATA LOCAL INFILE 'C:\\Users\\nicol\\OneDrive\\Desktop\\categories.csv'
INTO TABLE categories
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;


# ----------------> Products Table
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    ProductID     INT,
    ProductName   VARCHAR(100),
    Price     VARCHAR(50),
    CategoryID    INT,
    Class         VARCHAR(45),
    ModifyDate    VARCHAR(20),
    Resistant     VARCHAR(45),
    IsAllergic    VARCHAR(10),
    VitalityDays  INT
);

LOAD DATA LOCAL INFILE 'C:\\Users\\nicol\\OneDrive\\Desktop\\products.csv'
INTO TABLE products
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

UPDATE products
SET Price = REPLACE(REPLACE(Price, '$', ''), ',', '.');

UPDATE products
SET Price = ROUND(Price, 2);


# ----------------> Sales Table
DROP TABLE IF EXISTS sales;
CREATE TABLE sales (
    SaleID           INT,
    SalePersonID     INT,
    CustomerID       INT,
    ProductID        INT,
    Quantity         INT,
    Discount         DECIMAL(10,2),
    TotalPrice       DECIMAL(10,2),
    SaleDate         VARCHAR(30),
    TransactionNumber VARCHAR(255)
);
LOAD DATA LOCAL INFILE 'C:\\Users\\nicol\\OneDrive\\Desktop\\sales.csv'
INTO TABLE sales
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;

SET SQL_SAFE_UPDATES = 1;
``` 

___

TASK 2 (questions)
============================= 

**PREGUNTA 1**
- ¿Cuáles fueron los 5 productos más vendidos (por cantidad total)?
```
SELECT
  p.productid,
  p.productname,
  SUM(s.quantity) AS totalselled
FROM sales s
JOIN products p ON p.productID = s.productid
GROUP BY
  p.productid,
  p.productname
ORDER BY
  totalselled DESC
LIMIT 5;

```

- Para resolverlo cruzamos las tablas de Sales y Producto (productid), luego agrupamos por productid para calcular la suma de la cantidad total de productos vendidos.
- Aplicamos un order by Desc para tener una vista descendente de los productos mas vendidos y aplicando el Limit 5 logramos ver el siguiente resultado..

![primer rta](images/pa_q1.png)


**PREGUNTA 2**
- ¿Cuáles fueron los 5 productos más vendidos (por cantidad total), y cuál fue el vendedor que más unidades vendió de cada uno? Una vez obtenga los resultados, en el análisis responde: ¿Hay algún vendedor que aparece más de una vez como el que más vendió un producto? ¿Algunos de estos vendedores representan más del 10% de la ventas de este producto?

```
WITH 
# ----------------> Total selled by Product
sells_by_prod AS (
  SELECT 
	productid,
    SUM(quantity) AS total_selled
  FROM sales
  GROUP BY productid
  ),

# ---------------->  Total selled by Employee & Product
sells_by_employee AS (
  SELECT
    productid,
    salepersonid,
    SUM(quantity) AS total_selled
  FROM sales
  GROUP BY productid, salepersonid
),

# ---------------->  Ranking of Sellers
ranking_sellers AS (
  SELECT
    productid,
    salepersonid,
    total_selled,
    ROW_NUMBER() OVER (PARTITION BY productid ORDER BY total_selled DESC) AS rn
  FROM sells_by_employee
)

SELECT
  p.productid,
  p.productname,
  sbp.total_selled,
  CONCAT(e.firstname, ' ', e.lastname) AS topseller,
  rs.total_selled    AS sellerqty,
  ROUND(rs.total_selled / sbp.total_selled * 100, 2) AS sellerpct
FROM sells_by_prod sbp 
	JOIN products p ON p.productid = sbp.productid
	JOIN ranking_sellers rs ON rs.productid = sbp.productid AND rs.rn = 1
	JOIN employees e ON e.employeeid = rs.salepersonid
	ORDER BY sbp.total_selled DESC
	LIMIT 5;

```
- Primero calculamos el total vendido por producto (sells_by_prod).
- Despues sumamos lo vendido por cada vendedor y producto (sells_by_employee).
- Con ROW_NUMBER() creamos un ranking interno y filtramos rn = 1 para quedarnos solo con el top seller de cada producto.
- Finalmente unimos todo con las tablas de productos y empleados, ordenamos por el total vendido y limitamos a 5 para obtener los 5 productos mas vendidos junto a su vendedor estrella.

![segunda rta](images/pa_q2.png)


**PREGUNTA 3**
- Entre los 5 productos más vendidos, ¿cuántos clientes únicos compraron cada uno y qué proporción representa sobre el total de clientes? Analiza si ese porcentaje sugiere que el producto fue ampliamente adoptado entre los clientes o si, por el contrario, fue comprado por un grupo reducido que generó un volumen alto de ventas. Compara los porcentajes entre productos e identifica si alguno de ellos depende más de un segmento específico de clientes.

```
WITH sells_by_prod AS (
  SELECT 
    productid,
    SUM(quantity) AS total_selled
  FROM sales
  GROUP BY productid
),

top5_prods AS (
  SELECT
    productid
  FROM sells_by_prod
  ORDER BY total_selled DESC
  LIMIT 5
),

clients_per_prod AS (
  SELECT
    s.productid,
    COUNT(DISTINCT s.customerid) AS unique_clients
  FROM sales s
  JOIN top5_prods t
    ON s.productid = t.productid
  GROUP BY s.productid
),

total_clients AS (
  SELECT COUNT(*) AS total_clients
  FROM customers
)


SELECT
  p.productid,
  p.productname,
  cpp.unique_clients,
  ROUND(cpp.unique_clients / tc.total_clients * 100, 2) AS client_pct
FROM clients_per_prod cpp
JOIN products p ON p.productid = cpp.productid
CROSS JOIN total_clients tc
ORDER BY cpp.unique_clients DESC;

```
- Primero agrupamos ventas por producto para obtener total_selled (sells_by_prod).
- Luego elegimos los 5 productos con mayor total_selled (top5_prods).
- Despues contamos clientes distintos que compraron cada uno de esos 5 productos (clients_per_prod).
- Calculamos el numero total de clientes registrados (total_clients).
- Finalmente unimos todo, calculamos el porcentaje de clientes unicos por producto y ordenamos de mayor a menor.

![tercer rta](images/pa_q3.png)



**PREGUNTA 4**
- ¿A qué categorías pertenecen los 5 productos más vendidos y qué proporción representan dentro del total de unidades vendidas de su categoría? Utiliza funciones de ventana para comparar la relevancia de cada producto dentro de su propia categoría.


```
WITH prod_totals AS (
  SELECT
    productid,
    SUM(quantity) AS total_selled
  FROM sales
  GROUP BY productid
),
top5_prods AS (
  SELECT
    productid,
    total_selled
  FROM prod_totals
  ORDER BY total_selled DESC
  LIMIT 5
),
prod_with_cat AS (
  SELECT
    pt.productid,
    pt.total_selled,
    p.categoryid,
    SUM(pt.total_selled) OVER (PARTITION BY p.categoryid) AS category_total_selled
  FROM prod_totals pt
  JOIN products p ON p.productid = pt.productid
)
SELECT
  pwc.productid,
  p.productname,
  pwc.categoryid,
  c.categoryname,
  pwc.total_selled,
  pwc.category_total_selled,
  ROUND(pwc.total_selled / pwc.category_total_selled * 100, 2) AS product_pct_in_category
FROM prod_with_cat pwc
JOIN top5_prods t5 ON t5.productid = pwc.productid
JOIN products p ON p.productid = pwc.productid
JOIN categories c ON c.categoryid = pwc.categoryid
ORDER BY pwc.total_selled DESC;

```
- Primero sumamos ventas por producto para obtener total_selled (prod_totals).
- Luego extraemos los 5 productos con mayor total_selled (top5_prods).
- En prod_with_cat unimos con products y usamos SUM() OVER (PARTITION BY categoria) para calcular el total vendido por categoria.
- Finalmente filtramos solo esos 5 productos, los unimos con las tablas de products y categories, y calculamos el porcentaje que cada producto aporta al total de su categoria.
![cuarta rta](images/pa_q4.png)


**PREGUNTA 5**


- PREGUNTA 5: ¿Cuáles son los 10 productos con mayor cantidad de unidades vendidas en todo el catálogo y cuál es su posición dentro de su propia categoría? Utiliza funciones de ventana para identificar el ranking de cada producto en su categoría. Luego, analiza si estos productos son también los líderes dentro de sus categorías o si compiten estrechamente con otros productos de alto rendimiento. ¿Qué observas sobre la concentración de ventas dentro de algunas categorías?


```
WITH 

prod_totals AS (
  SELECT
    productid,
    SUM(quantity) AS total_selled
  FROM sales
  GROUP BY productid
),

prod_rank_category AS (
  SELECT
    pt.productid,
    pt.total_selled,
    p.categoryid,
    ROW_NUMBER() OVER ( PARTITION BY p.categoryid ORDER BY pt.total_selled DESC) AS cat_rank
  FROM prod_totals pt
  JOIN products p ON p.productid = pt.productid
)
  
SELECT
  prc.productid,
  p.productname,
  prc.total_selled,
  prc.categoryid,
  c.categoryname,
  prc.cat_rank
FROM prod_rank_category prc
JOIN products p ON p.productid = prc.productid
JOIN categories c ON c.categoryid = prc.categoryid
ORDER BY prc.total_selled DESC
LIMIT 10;

```

- Primero calculamos el total vendido por producto en prod_totals.
- Despues unimos con products y usamos ROW_NUMBER() OVER (PARTITION BY categoria) para darle un ranking (cat_rank) a cada producto dentro de su categoria.
- Finalmente seleccionamos los datos de prod_rank_category, los alimentamos con los nombres de producto y categoria, ordenamos por total_selled de mayor a menor y limitamos a 10 para ver los top 10 productos segun su performance.
![quinta rta](images/pa_q5.png)
